In [3]:
import json
import pydotplus as pdp
from IPython.display import Image  
from glob import glob

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Convert the dot files to json for the demo.

In [ ]:
for f in glob("f*/*.dot"):
    
    graph = pdp.graphviz.graph_from_dot_file(f)

    edges = sorted([(int(e.get_source()), int(e.get_destination())) for e in graph.get_edge_list()])
    nodes = sorted([int(n.get_label()) for n in graph.get_node_list() if n.get_label()])

    j = []
    for n in nodes:
        j.append({"id" : n, "nei" : []})
        for e in edges:
            if n in e:
                j[-1]["nei"].append(e[0] if e[1] == n else e[1])
                
    with open(f.replace("dot", "json"), "w") as out:
        json.dump(j, out)

This is just a closure check, that I "retrieve" the right diagrams.

In [ ]:
for f in glob("f*/*.json"):
    
    graph = pdp.Dot(graph_type='graph')

    with open(f) as i: 
        j = json.load(i)
        
    nodes = []
    for n in j:
        nodes.append(pdp.Node(str(n["id"])))
        graph.add_node(nodes[-1])

    for n in j:
        for e in n["nei"]:
            if e < n["id"]: continue
            graph.add_edge(pdp.Edge(nodes[e-1], nodes[n["id"]-1]))
        
        
    Image(graph.create_png(prog = "circo"))

In [ ]:
for f in glob("f*/*.dot"):
    
    graph = pdp.graphviz.graph_from_dot_file(f)
    Image(graph.create_png(prog = "neato"))

### Get all the graphs -- they're all set.

In [91]:
graphs = {}
for f in glob("f*/*.json"):
    with open(f) as i:
        graphs[f.replace(".json", "")] = json.load(i)

In [93]:
graphs["f5/a"]

[{'id': 1, 'nei': [2, 5, 8]},
 {'id': 2, 'nei': [1, 3, 6]},
 {'id': 3, 'nei': [2, 4, 7]},
 {'id': 4, 'nei': [3, 5, 8]},
 {'id': 5, 'nei': [1, 4, 6]},
 {'id': 6, 'nei': [2, 5, 7]},
 {'id': 7, 'nei': [3, 6, 8]},
 {'id': 8, 'nei': [4, 7, 1]}]

In [94]:
from random import random, shuffle

def iterate(graph = graphs["f2/a"], a = 0.100, b = 0.015, N = 100000):
    
    for n in graph:
        n["hist"] = []
        n["empl"] = True
        
    for i in range(N):
        
        for n in graph:
            n["hist"].append(n["empl"])

        for n1 in graph:
            if random() < a:
                if not n1["hist"][-1]:
                    n1["empl"] = True
                    
                else: # Pass to a neighbor:
                    shuffle(n1["nei"])
                    
                    found = False
                    for nei in n1["nei"]:
                        for n2 in graph:
                            if n2["id"] == nei and not n2["hist"][-1]:
                                n2["empl"] = True
                                found = True
                                break
                        if found: break
                                            
        for n in graph:
            if random() < b:
                n["empl"] = False
                

In [76]:
def unemployment(l): return 1 - sum(l)/len(l)

In [78]:
import math

def correlation(l1, l2):
    
    # Here, var is E[X^2] - E[X]^2.
    # Since 1^2 = 1, E[X^2] = E[X].

    avg1 = sum(l1)/len(l1)
    var1 = avg1 - avg1**2

    avg2 = sum(l2)/len(l2)
    var2 = avg2 - avg2**2
    
    exp = sum([(x1 - avg1)*(x2 - avg2) for x1, x2 in zip(l1, l2)]) / len(l1)
    
    # print(var1, var2)
    return exp / math.sqrt(var1 * var2)


### In Figure 1, agent 1 benefits from agent 3 in the long run (s₁(a) > s₁(b)).  But for s<sub>t-1</sub> = (0, 1, 0) as given, they are anticorrelated.

In [75]:
iterate(graphs["f1/a"], N = 10000000)
iterate(graphs["f1/b"], N = 10000000)

In [77]:
unemployment(graphs["f1/a"][0]["hist"])
unemployment(graphs["f1/b"][0]["hist"])

0.07901349999999996

0.07853089999999996

In [81]:
s1_hist = graphs["f1/a"][0]["hist"]
s2_hist = graphs["f1/a"][1]["hist"]
s3_hist = graphs["f1/a"][2]["hist"]

s1_cond = [x for xi, x in enumerate(graphs["f1/a"][0]["hist"])
           if xi and s1_hist[xi-1] == 0 and s2_hist[xi-1] == 1 and s3_hist[xi-1] == 0]

s3_cond = [x for xi, x in enumerate(graphs["f1/a"][2]["hist"])
           if xi and s1_hist[xi-1] == 0 and s2_hist[xi-1] == 1 and s3_hist[xi-1] == 0]

correlation(s1_cond, s3_cond)

-0.018294128881491654

### In Figure 2 A-D, the employment rates are 0.132, 0.083, 0.063, and 0.050.

In [ ]:
iterate(graphs["f2/a"], N = 10000000)
iterate(graphs["f2/b"], N = 10000000)
iterate(graphs["f2/c"], N = 10000000)
iterate(graphs["f2/d"], N = 10000000)

In [67]:
unemployment(graphs["f2/a"][0]["hist"])
unemployment(graphs["f2/b"][0]["hist"])
unemployment(graphs["f2/c"][0]["hist"])
unemployment(graphs["f2/d"][0]["hist"])

0.13240249999999998

0.07800220000000002

0.05655410000000005

0.04649650000000005

No, but their footnotes 8 and 9 are not encouraging:

_The numbers for more than one agent are obtained from simulations in Maple􏰥. We simulate the economy over a large number of periods (hundreds of thousands) and calculate observed unemployment averages and correlations. The programs are available upon request from the authors. The correlation numbers are only moderately accurate, even after several hundred thousand periods._

_In some cases, the correlations are indistinguishable to the accuracy of our simulations._

If I use as few iterations, I can reproduce their numbers.

### The Corr(s₁ s₂) are --, 0.041, 0.025, and 0.025.

In [29]:
correlation(graphs["f2/a"][0]["hist"], graphs["f2/a"][1]["hist"])
correlation(graphs["f2/b"][0]["hist"], graphs["f2/b"][1]["hist"])
correlation(graphs["f2/c"][0]["hist"], graphs["f2/c"][1]["hist"])
correlation(graphs["f2/d"][0]["hist"], graphs["f2/d"][1]["hist"])

-0.0071912270432927155

0.034276845524049464

0.03273460330759649

0.018672164712226862

### Figure 4: The employment rates for types 1, 2, and 3 are 0.047, 0.048, and 0.050.

In [82]:
iterate(graphs["f4/a"], N = 10000000)

unemp_rates = {}
for node in range(10):
    unemp_rates[node+1] = unemployment(graphs["f4/a"][node]["hist"])
    print("{:d} :: {:.4f}".format(node+1, unemp_rates[node+1]))

(unemp_rates[1] + unemp_rates[6])/2
(unemp_rates[2] + unemp_rates[5] + unemp_rates[7] + unemp_rates[10])/4
(unemp_rates[3] + unemp_rates[4] + unemp_rates[8] + unemp_rates[9]) /4

1 :: 0.0454
2 :: 0.0456
3 :: 0.0460
4 :: 0.0461
5 :: 0.0460
6 :: 0.0455
7 :: 0.0458
8 :: 0.0461
9 :: 0.0460
10 :: 0.0456


0.04544490000000001

0.045725425000000014

0.04603145

### The average path length matters for unemployment

We can get the path lengths right, but the employment rates are slightly lower than they quote, and only very slightly different.

In [86]:
def avg_path_length_single(node, graph):
    
    max_d = 0
    dist  = {node : 0}
    nodes = {n["id"] for n in graph}
    
    while max_d in dist.values():
        for n in graph:
            if n["id"] in dist and dist[n["id"]] == max_d:
                for nei in n["nei"]:
                    if nei not in dist:
                        dist[nei] = max_d+1
            
        max_d += 1
        
        # print(node, max_d, [k for k, v in dist.items() if v == max_d])
    
    if len(nodes) > len(dist):
        return float('inf')
    else:
        return sum(dist.values())/(len(dist)-1)
    
    
def avg_path_length(graph = graphs["f5/a"]):
    
    return sum([avg_path_length_single(n["id"], graph) for n in graph])/len(graph)
    
    
avg_path_length(graphs["f5/a"]), avg_path_length(graphs["f5/b"])

(1.5714285714285714, 1.7857142857142856)

In [85]:
iterate(graphs["f5/a"], N = 5000000)
iterate(graphs["f5/b"], N = 5000000)

sum([unemployment(graphs["f5/a"][n]["hist"]) for n in range(8)])/8
sum([unemployment(graphs["f5/b"][n]["hist"]) for n in range(8)])/8

0.045372625

0.045636575

### §III, Fig. 6: for connected agents, expectations of next-round employment fall with length of unemployment duration, because duration is indicative of peers current status.

In [87]:
def conditional_prob_periods(hist, periods):
    
    cond = []
    for xi in range(periods, len(hist)):
        if not any([hist[xpi] for xpi in range(xi-periods, xi)]):
            cond.append(hist[xi])
            
    return sum(cond)/len(cond)

conditional_prob_periods(graphs["f2/b"][0]["hist"], 1)
conditional_prob_periods(graphs["f2/b"][0]["hist"], 2)
conditional_prob_periods(graphs["f2/b"][0]["hist"], 10)
conditional_prob_periods(graphs["f2/b"][0]["hist"], 20)

conditional_prob_periods(graphs["f2/d"][0]["hist"], 1)
conditional_prob_periods(graphs["f2/d"][0]["hist"], 2)
conditional_prob_periods(graphs["f2/d"][0]["hist"], 10)
conditional_prob_periods(graphs["f2/d"][0]["hist"], 20)

0.17641169095230647

0.17562580068709266

0.16915588556205807

0.1603089875450245

0.30672416203370145

0.30213526333259916

0.2675789376358005

0.2674897119341564